# Desafio complementar

In [115]:
import pandas as pd 
import requests
import numpy as np
from plyer import notification
from datetime import datetime

## 00. Funções auxiliares

In [64]:
def alerta(nivel, base, etapa):
    """
    Retorna um alerta no desktop

    :param nivel: Nível do alerta
    :param base: Base onde ocorreu a falha
    :param etapa: Etapa onde ocorreu a falha

    :type nivel: int
    :type base: str
    :type etapa: str
    """

    niveis = {1:"Baixo",
          2: "Médio",
          3: "Alto"}

    if nivel not in niveis:
        raise ValueError(f"Nível inválido. Deve ser 1, 2 ou 3")


    return notification.notify(title=f"ATENÇÃO: Alerta {niveis[nivel]}",
                               message=f"Falha no carregamento da base {base} na etapa {etapa}\n{datetime.now()}")

def get_req(url, base="base"):
    """
    Retorna o JSON de uma requisição a uma API. Em caso de resposta diferente de 200, retorna um alerta.

    :param url: URL da API
    :param base: Nome da base que será criada

    :type url: str
    :type base: str
    """
    # Trata respostas diferentes da esperada (200)
    req = requests.get(url)
    if req.status_code != 200:
        return alerta(1, base, "Carregamento")
    
    else:
        return req.json()



## 01. Premissas para extração
Para esta atividade, utilizaremos a API REST Countries

Vamos extrair 3 tabelas a partir da requisição

In [71]:
url = "https://restcountries.com/v3.1/all"
req = get_req(url)

In [88]:
print(f"Entradas na tabela: {len(req)}")

dados_disponiveis = list(req[0].keys())
print(f"Dados disponíveis: {dados_disponiveis}")



Entradas na tabela: 250
Dados disponíveis: ['name', 'tld', 'cca2', 'ccn3', 'cca3', 'cioc', 'independent', 'status', 'unMember', 'currencies', 'idd', 'capital', 'altSpellings', 'region', 'subregion', 'languages', 'translations', 'latlng', 'landlocked', 'area', 'demonyms', 'flag', 'maps', 'population', 'gini', 'fifa', 'car', 'timezones', 'continents', 'flags', 'coatOfArms', 'startOfWeek', 'capitalInfo', 'postalCode']


In [116]:
names = [i["name"]["common"] for i in req]
regions = [i["region"] for i in req]
#Tratamos o caso de não existir a chave "subregion" no JSON da requisição
subregions = [i["subregion"] if "subregion" in i else np.nan for i in req]


In [135]:
df = pd.DataFrame({"name": names,
                   "region": regions,
                   "subregion": subregions})
rows_with_nan = df.isna().any(axis=1)
df[rows_with_nan]

,name,region,subregion
102,French Southern and Antarctic Lands,Antarctic,NaN
159,Bouvet Island,Antarctic,NaN
185,South Georgia,Antarctic,NaN
194,Antarctica,Antarctic,NaN
218,Heard Island and McDonald Islands,Antarctic,NaN


In [111]:
j=1
for i in req:
    print (f'{j}: {i["subregion"]}')
    j += 1

1: Southern Europe
2: Eastern Africa
3: Western Africa
4: North America
5: Southern Europe
6: Polynesia
7: Eastern Africa
8: Eastern Africa
9: South America
10: Central Asia
11: Southeast Europe
12: Southeast Europe
13: Northern Europe
14: Northern Africa
15: South-Eastern Asia
16: Middle Africa
17: Western Asia
18: Eastern Africa
19: Polynesia
20: Western Africa
21: Western Africa
22: South-Eastern Asia
23: Central America
24: Southern Asia
25: Caribbean
26: Northern Europe
27: Central America
28: Western Asia
29: Middle Africa
30: Central Asia
31: Central Europe
32: Micronesia
33: Western Africa
34: Northern Europe
35: Western Asia
36: Northern Europe
37: Caribbean
38: Central Asia
39: Eastern Africa
40: Northern Africa
41: Northern Africa
42: Caribbean
43: Australia and New Zealand
44: Australia and New Zealand
45: South-Eastern Asia
46: South America
47: Polynesia
48: Southeast Europe
49: Northern Europe
50: Eastern Europe
51: Central America
52: Middle Africa
53: Central Europe
54

KeyError: 'subregion'